In [22]:

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
# imprt tree
from sklearn import tree
# import metrics
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder

In [23]:
df_raw = pd.read_csv("./labeled_data/labeled_df_15.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",", index_col=False,
                    usecols=['SessionID','Activity','anomaly'])
df_raw

,SessionID,Activity,anomaly
0,46,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
1,46,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,False
2,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,False
3,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
4,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
...,...,...,...
7174929,55314751,/werk_nl/werknemer/werkmap,False
7174930,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
7174931,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
7174932,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False


In [24]:
# change anomaly == True to 1 and False to 0
df_raw["anomaly"] = df_raw["anomaly"].astype(int)

In [25]:
# count the number of unique SessionID where anomaly == True
count_anomaly_raw = df_raw[df_raw["anomaly"] == 1]["SessionID"].nunique()
count_normal_raw = df_raw[df_raw["anomaly"] == 0]["SessionID"].nunique()
print(f'Amount of anomalous sessions in the dataset:   {count_anomaly_raw}' )
print(f'Amount of normal sessions in the dataset:      {count_normal_raw}')
print(f'total sessions in the dataset:                 {count_anomaly_raw + count_normal_raw}')
distribution =  count_anomaly_raw / count_normal_raw 
print(f'Distribution:                                  {distribution * 100} %')

Amount of anomalous sessions in the dataset:   355
Amount of normal sessions in the dataset:      659915
total sessions in the dataset:                 660270
Distribution:                                  0.05379480690695014 %


In [26]:
df_anomaly = df_raw[df_raw["anomaly"] == 1].copy()
df_normal = df_raw[df_raw["anomaly"] == 0].copy()
nr_of_sessions_used = 50000
injection_rate = nr_of_sessions_used / count_normal_raw
injection_amount = int(injection_rate * count_anomaly_raw)
print(injection_rate)
print(injection_amount)
# get 20 random sessionIDs from the anomaly dataset
anomaly_sessionIDs = df_anomaly["SessionID"].sample(n=injection_amount, random_state=1).tolist()

0.07576733367176075
26


In [27]:
df_ = df_normal[df_normal["SessionID"].isin(df_normal["SessionID"].unique()[:nr_of_sessions_used])].copy()

df_ = df_.append(df_anomaly[df_anomaly["SessionID"].isin(anomaly_sessionIDs)], ignore_index=True)

# Only use the columns that are needed -> SessionID, Activity, anomaly
df_50k = df_[["SessionID", "Activity", "anomaly"]]
print(df_50k["SessionID"].nunique())
# remove the sessions with ID in anomaly_sessionIDs from the df_anomaly dataset
df_anomaly = df_anomaly[~df_anomaly["SessionID"].isin(anomaly_sessionIDs)].copy()
df_anomaly.to_csv("gen_sessions/an.csv", index=False)


50026


In [20]:
# count the number of unique SessionID where anomaly == True
count_anomaly = df_50k[df_50k["anomaly"] == 1]["SessionID"].nunique()
count_normal = df_50k[df_50k["anomaly"] == 0]["SessionID"].nunique()
print(f'Amount of anomalous sessions in the sampled dataset:   {count_anomaly}')
print(f'Amount of normal sessions in the sampled dataset:      {count_normal}')
distribution =  count_anomaly / count_normal 
print(f'Distribution:                                          {distribution * 100} %')
# unique SessionIDs
df_50k["SessionID"].nunique()

Amount of anomalous sessions in the sampled dataset:   26
Amount of normal sessions in the sampled dataset:      50000
Distribution:                                          0.052 %


50026

Plan:
- Het gaat om de transities tussen de verschillende activiteiten
  1. Encode alle transities
  2. Maak een dictionary met alle transities




In [31]:
#  Helper function
# function to create a list of all the transitions
def get_all_transitions(_df):
  df = _df.copy()
  transitions = []
  # create consecutive column
  df['consecutive'] = df.groupby("SessionID")['Activity'].shift(-1)
  # if consecutive is NaN, then consecutive = END
  df['consecutive'] = df['consecutive'].fillna("END")
    # create column with the transition
  df['transition'] = df['Activity'] + "->" + df['consecutive']
  # get all unique transitions en put in transitions list
  transitions = df['transition'].unique().tolist()
  return transitions

# function that add a column 'transition' to the df with activity and consecutive
def create_transition_df(_df:pd.DataFrame) -> pd.DataFrame:
  df = _df.copy()
  # create consecutive column
  df['consecutive'] = df.groupby("SessionID")['Activity'].shift(-1)
  # if consecutive is NaN, then consecutive = END
  df['consecutive'] = df['consecutive'].fillna("END")
    # create column with the transition
  df['transition'] = df['Activity'] + "->" + df['consecutive']
  # drop activity and consecutive
  df = df.drop(columns=["Activity", "consecutive"])
  return df


# function that combines a item in a list with the next item
def list_combine(_list):
  list_ = _list.copy()
  res= []
  for index,item in enumerate(list_):
    if index < len(list_)-1:
      res.append(item + "->" + list_[index+1])
    else:
      res.append(item + "->" + "END")
  return res

# function to transform each session into a sequence of activities
def transform_session_to_sequence(_df):
  df = _df.copy()
  df_ses = df.groupby("SessionID")["transition"].apply(list).reset_index()
  # merge the anomaly column
  df_ses = df_ses.merge(df[["SessionID", "anomaly"]].drop_duplicates(), on="SessionID")
  return df_ses

  
# function that creates a empty dataframe where each column is a transition from the transition list
def create_tree_df(transitions:list) -> pd.DataFrame:
  return pd.DataFrame(columns=transitions)

  
def merge_tree_and_transitions(_tree_df:pd.DataFrame, _transitions_df:pd.DataFrame) -> pd.DataFrame:
  tree_df = _tree_df.copy()
  transitions_df = _transitions_df.copy()
  # change all transitions to a serie for each session
  session = transitions_df.groupby("SessionID")["transition"].apply(list)  
  return session


def transition_count(_df):
  # function that counts the number of times a all transitions occurs in a session
  df = _df.copy()
  df = df.groupby("SessionID")["transition"].value_counts().unstack().fillna(0)
  # df = df.groupby("SessionID")["transition"].sum().unstack().fillna(0)
  # add the anomaly column
  df['anomaly'] = df_.groupby("SessionID")["anomaly"].max()
  return df
  

In [32]:
#  GLOBALS
ALL_TRANSITIONS = get_all_transitions(df_raw)
k_transitions = get_all_transitions(df_50k)
print(len(ALL_TRANSITIONS))
print(len(k_transitions))

15947
5160


In [33]:
df_trans = create_transition_df(df_50k)

base_data = transition_count(df_trans)

In [34]:
base_data

transition,/->/werk_nl/werknemer/home,/->/werk_nl/werknemer/mijn_werkmap,/->/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn_sollicitaties,/->/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/->/werk_nl/werknemer/mijn_werkmap/postvak/mijn_berichten,/->/werk_nl/werknemer/mijn_werkmap/postvak/mijn_documenten,/->/werk_nl/werknemer/mijn_werkmap/postvak/mijn_tips,/->/werk_nl/werknemer/mijn_werkmap/werk-zoeken/mijn_cv,/->/werk_nl/werknemer/mijn_werkmap/werk-zoeken/vacatures_bij_mijn_cv,/->/werk_nl/werknemer/mijn_werkmap/werk-zoeken/vacatures_zoeken,...,/zoeken_portlet/ajax/zoekBeroep->/werk_nl/werknemer/mijn_werkmap/werk-zoeken/vacatures_zoeken,/zoeken_portlet/ajax/zoekBeroep->/werk_nl/werknemer/ontslag/transitievergoeding,/zoeken_portlet/ajax/zoekBeroep->/werk_nl/werknemer/solliciteren,/zoeken_portlet/ajax/zoekBeroep->/werk_nl/werknemer/vacatures,/zoeken_portlet/ajax/zoekBeroep->/werk_nl/werknemer/werkmap,/zoeken_portlet/ajax/zoekBeroep->/werk_nl/werknemer/zoeken,/zoeken_portlet/ajax/zoekBeroep->/zoeken_portlet/ajax/zoekAantalIndicatief,/zoeken_portlet/ajax/zoekBeroep->/zoeken_portlet/ajax/zoekBeroep,/zoeken_portlet/ajax/zoekBeroep->END,anomaly
SessionID,,,,,,,,,,,,,,,,,,,,,
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42771301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
45419685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
46475745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1


In [35]:
# Function to split the data into train and test data
def split_data(_df):
  df = _df.copy()
  X = df.drop(columns=["anomaly"])
  y = df["anomaly"]
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

In [36]:
# Base test and train data
X_train, X_test, y_train, y_test = split_data(base_data)

In [37]:
clf = tree.DecisionTreeClassifier().fit(X_train, y_train)

In [38]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score


clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
_y_predict = clf.predict(X_test)
print(accuracy_score(y_test, _y_predict))
print(balanced_accuracy_score(y_test, _y_predict))


0.9999000599640215
0.9


In [17]:

# import the generated sessions:
ses_amount = 1000
base_path = f"gen_sessions/{str(ses_amount)}/"

gen_sessions_paths = [
  base_path + f'5_{ses_amount}.csv',
  base_path + f'10_{ses_amount}.csv',
  base_path + f'25_{ses_amount}.csv',
  base_path + f'50_{ses_amount}.csv',
  base_path + f'75_{ses_amount}.csv',
  base_path + f'100_{ses_amount}.csv',
  # base_path + 'an.csv',
  
  # base_path + '75_10000.csv',
  # base_path + '100_10000.csv',
]

In [18]:
auc_score_dicts = {}
dict_prec_and_rec_list = {}

amount_anomalies_list_total = []
precision_score_list_total = []
recall_score_list_total = []

In [19]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score


for sessions in gen_sessions_paths:
    amount_anomalies_list = []
    precision_score_list = []
    recall_score_list = []
    
    # build the dataFrame
    cvs = pd.read_csv(sessions,index_col=False)
    # check if column Unnamed: 0 exists
    if "Unnamed: 0" in cvs.columns : cvs = cvs.drop(columns=["Unnamed: 0"])
    # rename 
    cvs = cvs.rename(columns={'URL_FILE':'Activity'})
    
    gen_transitions = create_transition_df(cvs)

    ready_df = transition_count(gen_transitions)
    ready_df['anomaly'] = 1
    
    for amount_gen in range(0, 100,10):
        amount_real = 0
        df_gen = ready_df[:amount_gen]
         
        # Add the generated anomalies to the training dataset
        X_train_extra = pd.concat([X_train, df_gen.drop(columns=["anomaly"])]).fillna(0)
        y_train_extra = pd.concat([y_train, df_gen["anomaly"]])
        
        
        #Make sure that both dataframes have the same columns
        for column_name in X_train_extra.columns:
            if column_name not in X_test.columns:
                X_test[column_name] = 0

        #Make sure that both dataframes have the same columns
        # for column_name in X_test.columns:
        #     if column_name not in X_train_extra.columns:
        #         X_test.drop(columns=[column_name], inplace=True)
        
        
        clf = tree.DecisionTreeClassifier()
        clf.fit(X_train_extra, y_train_extra)
       
        y_test_predict = clf.predict(X_test)
        
        #AUC predict
        print(sessions, amount_gen)
        print(accuracy_score(y_test_predict, predictions))
        print(balanced_accuracy_score(y_test_predict, predictions))
        print()
        
    #     #AUC predict
    #     fpr, tpr, thresholds = metrics.roc_curve(y_test, y_test_predict)
    #     print(sessions, amount_gen)
    #     print(metrics.auc(fpr, tpr))
    #     print()
    #     auc_score_dicts[(sessions, amount_gen)] = metrics.auc(fpr, tpr)
        

        
    #     precision_score = [i for i in classification_report(y_test, y_test_predict).split(" ") if len(i) > 2][8]
    #     recall_score = [i for i in classification_report(y_test, y_test_predict).split(" ") if len(i) > 2][9]

    #     amount_anomalies_list.append(amount_gen)
    #     precision_score_list.append(precision_score)
    #     recall_score_list.append(recall_score)
       
        
    # amount_anomalies_list = [int(i) for i in amount_anomalies_list]
    # precision_score_list = [float(i) for i in precision_score_list]
    # recall_score_list = [float(i) for i in recall_score_list]
    # print(amount_anomalies_list, precision_score_list, recall_score_list)
    
    # string_recall = "recall_" + str(sessions)
    # string_precision = "precision_" + str(sessions)
    # dict_prec_and_rec_list[string_recall] = recall_score_list
    # dict_prec_and_rec_list[string_precision] = precision_score_list
    
    # plt.figure(figsize=(10, 6))
    # plt.plot(amount_anomalies_list, precision_score_list, label = "precision_score")
    # plt.plot(amount_anomalies_list, recall_score_list, label = "recall_score")
    # plt.xlabel("Number of anomalous sessions injected in training data")
    # plt.ylabel("Score")
    # # save_string = "prec_rec_scores/" + str(sessions.split(".")[0].split("/")[1] + sessions.split(".")[0].split("/")[2]) + ".png"    
    # plt.legend()
    # # plt.savefig(save_string)
    # plt.show()
    

gen_sessions/1000/5_1000.csv 0
1.0
1.0

gen_sessions/1000/5_1000.csv 10
1.0
1.0



KeyboardInterrupt: 